Binary Network

In [5]:
import random

In [6]:
class Node:
    input_dim: int
    w: list[bool]
    b: int

    def __init__(self, input_dim: int, w: list[bool] | None = None, b: int | None = None):
        self.input_dim = input_dim

        if w is not None:
            self.w = w
        else:
            self._initialise_w()

        if b is not None:
            self.b = b
        else:
            self._initialise_b()

        self._check_w()
        self._check_b()

    def _initialise_w(self):
        self.w = [bool(random.getrandbits(1)) for _ in range(self.input_dim)]

    def _initialise_b(self):
        self.b = random.randint(0, self.input_dim)

    def _check_w(self):
        assert len(self.w) == self.input_dim

        for w_ in self.w:
            assert isinstance(w_, bool)

    def _check_b(self):
        assert isinstance(self.b, int)

        assert self.input_dim >= self.b >= 0

    def forward(self, x: list[bool]) -> bool:
        # inner product
        y_ = 0
        for w_, x_ in zip(self.w, x):
            y_ += w_ * x_
        # threshold
        y = y_ >= self.b
        return y

In [7]:
class Layer:
    input_dim: int
    output_dim: int
    nodes: list[Node]

    def __init__(self, input_dim: int, output_dim: int, nodes: list[Node] | None = None):
        self.input_dim = input_dim
        self.output_dim = output_dim

        if nodes is not None:
            self.nodes = nodes
        else:
            self._initialise_nodes()

        self._check_nodes()

    def _initialise_nodes(self):
        self.nodes = [Node(input_dim=self.input_dim) for _ in range(self.output_dim)]

    def _check_nodes(self):
        for node in self.nodes:
            assert node.input_dim == self.input_dim
        assert len(self.nodes) == self.output_dim

    def forward(self, x: list[bool]) -> list[bool]:
        return [node.forward(x) for node in self.nodes]

In [8]:
class Network:
    input_dim: int
    output_dim: int
    layers: list[Layer]

    def __init__(self, layers: list[Layer]):
        self.layers = layers

        self._check_layers()

        self.input_dim = self.layers[0].input_dim
        self.output_dim = self.layers[-1].output_dim

    def _check_layers(self):
        for input_layer, output_layer in zip(self.layers[:-1], self.layers[1:]):
            assert input_layer.output_dim == output_layer.input_dim

    def forward(self, x: list[bool]) -> list[bool]:
        for layer in self.layers:
            x = layer.forward(x)

        return x

    def __call__(self, x: list[bool]) -> list[bool]:
        return self.forward(x)

Toy Network

In [9]:
layer_1 = Layer(input_dim=2, output_dim=5)
layer_2 = Layer(input_dim=5, output_dim=5)
layer_3 = Layer(input_dim=5, output_dim=1)

In [10]:
network = Network([layer_1, layer_2, layer_3])

In [11]:
x_in = [True, False]

In [12]:
network(x_in)

[False]

Training Data - AND

In [13]:
train_input = [
    [False, False],
    [False, True],
    [True, False],
    [True, True],
]

train_output = [[x1 and x2] for x1, x2 in train_input]